**Importing packages**

In [17]:
import cv2
import mediapipe as mp
import os

**Parameters**

In [18]:
input_folder = "images" 
output_folder = "processed" 
blur_strength = 101          

*Initialize **Mediapipe** from OpenAI for face detection*

In [19]:
mp_face = mp.solutions.face_detection

**Main Function**

*This script loops through all images in the input folder, reads each image, and uses MediaPipe Face Detection to find faces. For every detected face, it calculates the bounding box, ensures it stays within image boundaries, extracts the face region, and applies Gaussian blur only to that area. The blurred face is then placed back into the image, and the processed result is saved with the same filename in the output folder, leaving the original untouched.*

In [21]:
with mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
    # Loop all images in images folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Can't read {filename}! Skipping!")
                continue

            processed = image.copy()
            results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if results.detections:
                for detection in results.detections:
                    bboxC = detection.location_data.relative_bounding_box
                    ih, iw, _ = image.shape
                    x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)

                    # The ROI is inside the image!
                    x, y = max(0, x), max(0, y)
                    w, h = min(iw - x, w), min(ih - y, h)

                    # Blurring the ROI
                    roi = processed[y:y+h, x:x+w]
                    roi_blur = cv2.GaussianBlur(roi, (blur_strength, blur_strength), 30)
                    processed[y:y+h, x:x+w] = roi_blur

            # Save image to process folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, processed)
            print(f"Processed : {output_path}")

Processed : process\1.jpg
Processed : process\2.jpg
Processed : process\3.jpg
Processed : process\4.jpg
Processed : process\5.jpg
Processed : process\6.jpg
Processed : process\7.jpg
Processed : process\8.jpg
